### Create a Personal Expense Tracker application 

- incorporating Pandas and NumPy for data analysis.
- Tkinter for the GUI.
- SQLite for the database. 
- This project will allow users to input, manage, and analyze their expenses. 
- It will include features like categorizing expenses, generating reports, and visualizing spending patterns.

In [ ]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Database setup
def create_db():
    conn = sqlite3.connect('expense_tracker.db')
    cursor = conn.cursor()

    cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT UNIQUE NOT NULL,
                        password TEXT NOT NULL)''')

    cursor.execute('''CREATE TABLE IF NOT EXISTS expenses (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        user_id INTEGER,
                        category TEXT,
                        amount REAL,
                        date TEXT,
                        description TEXT,
                        FOREIGN KEY (user_id) REFERENCES users (id))''')

    conn.commit()
    conn.close()

create_db()

# GUI setup
def login():
    username = username_entry.get()
    password = password_entry.get()
    conn = sqlite3.connect('expense_tracker.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM users WHERE username = ? AND password = ?', (username, password))
    user = cursor.fetchone()
    conn.close()
    if user:
        messagebox.showinfo("Login Success", "Welcome, " + username)
        show_main_app(user[0])
    else:
        messagebox.showerror("Login Failed", "Invalid username or password")

def register():
    def save_user():
        username = reg_username_entry.get()
        password = reg_password_entry.get()
        conn = sqlite3.connect('expense_tracker.db')
        cursor = conn.cursor()
        try:
            cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
            conn.commit()
            messagebox.showinfo("Registration Success", "User registered successfully")
            reg_window.destroy()
        except sqlite3.IntegrityError:
            messagebox.showerror("Registration Failed", "Username already exists")
        conn.close()

    reg_window = tk.Toplevel(root)
    reg_window.title("Register")

    tk.Label(reg_window, text="Username").grid(row=0)
    tk.Label(reg_window, text="Password").grid(row=1)

    reg_username_entry = tk.Entry(reg_window)
    reg_password_entry = tk.Entry(reg_window, show="*")

    reg_username_entry.grid(row=0, column=1)
    reg_password_entry.grid(row=1, column=1)

    tk.Button(reg_window, text="Register", command=save_user).grid(row=2, columnspan=2)

def add_expense(user_id, category, amount, date, description):
    conn = sqlite3.connect('expense_tracker.db')
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO expenses (user_id, category, amount, date, description)
                      VALUES (?, ?, ?, ?, ?)''', (user_id, category, amount, date, description))
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", "Expense added successfully")

def fetch_expenses(user_id):
    conn = sqlite3.connect('expense_tracker.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM expenses WHERE user_id = ?', (user_id,))
    expenses = cursor.fetchall()
    conn.close()
    return expenses

def analyze_expenses(expenses):
    df = pd.DataFrame(expenses, columns=['id', 'user_id', 'category', 'amount', 'date', 'description'])
    df['date'] = pd.to_datetime(df['date'])
    total_expense = df['amount'].sum()
    avg_monthly_expense = df.resample('M', on='date')['amount'].sum().mean()
    category_expense = df.groupby('category')['amount'].sum()
    return total_expense, avg_monthly_expense, category_expense

def plot_expenses_by_category(category_expense):
    category_expense.plot(kind='bar')
    plt.title('Expenses by Category')
    plt.xlabel('Category')
    plt.ylabel('Total Expense')
    plt.show()

def show_main_app(user_id):
    main_app = tk.Toplevel(root)
    main_app.title("Expense Tracker")

    def add_expense_gui():
        category = category_entry.get()
        amount = float(amount_entry.get())
        date = date_entry.get()
        description = description_entry.get()
        add_expense(user_id, category, amount, date, description)

    def analyze_and_show():
        expenses = fetch_expenses(user_id)
        total, avg_monthly, by_category = analyze_expenses(expenses)
        messagebox.showinfo("Analysis", f"Total Expense: {total}\nAverage Monthly Expense: {avg_monthly}")
        plot_expenses_by_category(by_category)

    tk.Label(main_app, text="Category").grid(row=0)
    tk.Label(main_app, text="Amount").grid(row=1)
    tk.Label(main_app, text="Date (YYYY-MM-DD)").grid(row=2)
    tk.Label(main_app, text="Description").grid(row=3)

    category_entry = tk.Entry(main_app)
    amount_entry = tk.Entry(main_app)
    date_entry = tk.Entry(main_app)
    description_entry = tk.Entry(main_app)

    category_entry.grid(row=0, column=1)
    amount_entry.grid(row=1, column=1)
    date_entry.grid(row=2, column=1)
    description_entry.grid(row=3, column=1)

    tk.Button(main_app, text="Add Expense", command=add_expense_gui).grid(row=4, columnspan=2)
    tk.Button(main_app, text="Analyze Expenses", command=analyze_and_show).grid(row=5, columnspan=2)

root = tk.Tk()
root.title("Personal Expense Tracker")

tk.Label(root, text="Username").grid(row=0)
tk.Label(root, text="Password").grid(row=1)

username_entry = tk.Entry(root)
password_entry = tk.Entry(root, show="*")

username_entry.grid(row=0, column=1)
password_entry.grid(row=1, column=1)

tk.Button(root, text="Login", command=login).grid(row=2, column=0)
tk.Button(root, text="Register", command=register).grid(row=2, column=1)

root.mainloop()
